In [1]:
import scanpy as sc
import pandas as pd
import sys
import numpy as np
import re

In [5]:
#Read in object
input_dir = "../data/current/publication/20220105/cellranger_reports/"
output_dir = "../data/current/publication/20220105/for_cellxgene"

input_files = [#"Adipocytes.h5ad", 
    "Cardiomyocytes2.h5ad"#,
    #"Endothelialcells.h5ad", 
    # "Fibroblasts.h5ad", 
    # "Global_all.h5ad", "Lymphoids.h5ad", "Macrophages.h5ad", 
    #"Muralcells_NEW.h5ad"
    ] #, "Neuronal_cells.h5ad"]

raw_input_file = "../data/current/annotated_expression/global_all_ANNOTATED_CELLSTATES_RAW.h5ad"
# output_file = "../data/current/publication/dcm_cell_atlas_for_cellxgene.h5ad"
meta_data = "../data/current/annotated_expression/global_all_ANNOTATED_CELLSTATES_RAW_V8_OBS_LONG.csv"

This object holds the full raw counts which are also required for upload to cellxgene

In [9]:
raw_obj = sc.read("C:/Users/Cem/Documents/GitHub/Helmholtz-Workspace/h5ad_formatter/local.h5ad")

In [12]:
deneme_obj = sc.read_h5ad("C:/Users/Cem/Documents/GitHub/Helmholtz-Workspace/h5ad_formatter/local.h5ad")

Use the adipocytes as an example to test things, because it is small. This object has the embeddings etc.

In [4]:
new_obj = sc.read("../data/current/publication/20220105/Adipocytes.h5ad")

In [13]:
raw_obj.obs.head()

,n_genes,n_counts,Batch,Donor_ID,Sample_ID,SARS-CoV-2_PCR,Group,Annotation,assay_ontology_term_id,cell_type_ontology_term_id,...,sex_ontology_term_id,tissue_ontology_term_id,cell_type,assay,disease,organism,sex,tissue,ethnicity,development_stage
RV8919578_AAACCTGAGAAACCTA-1,1159,2714.0,1,CV19-I-O-3,RV8919578,Positive,Multiple Sclerosis,NK CD56(bright),EFO:0011025,CL:0000938,...,unknown,UBERON:0000178,"CD16-negative, CD56-bright natural killer cell...",10x 5' v1,COVID-19,Homo sapiens,unknown,blood,unknown,unknown
RV8919578_AAACCTGAGAGCAATT-1,2058,7853.0,1,CV19-I-O-3,RV8919578,Positive,Multiple Sclerosis,Naïve B,EFO:0011025,CL:0000788,...,unknown,UBERON:0000178,naive B cell,10x 5' v1,COVID-19,Homo sapiens,unknown,blood,unknown,unknown
RV8919578_AAACCTGAGAGCTGGT-1,2597,10386.0,1,CV19-I-O-2,RV8919578,Positive,Multiple Sclerosis,pDC,EFO:0011025,CL:0000784,...,unknown,UBERON:0000178,plasmacytoid dendritic cell,10x 5' v1,COVID-19,Homo sapiens,unknown,blood,unknown,unknown
RV8919578_AAACCTGAGCGAAGGG-1,1902,5645.0,1,CV19-I-O-3,RV8919578,Positive,Multiple Sclerosis,NK CD56(dim),EFO:0011025,CL:0000939,...,unknown,UBERON:0000178,"CD16-positive, CD56-dim natural killer cell, h...",10x 5' v1,COVID-19,Homo sapiens,unknown,blood,unknown,unknown
RV8919578_AAACCTGAGCGATGAC-1,882,3754.0,1,CV19-I-O-2,RV8919578,Positive,Multiple Sclerosis,NK CD56(bright),EFO:0011025,CL:0000938,...,unknown,UBERON:0000178,"CD16-negative, CD56-bright natural killer cell...",10x 5' v1,COVID-19,Homo sapiens,unknown,blood,unknown,unknown


In [11]:
raw_obj.obs.columns

Index(['n_genes', 'n_counts', 'Batch', 'Donor_ID', 'Sample_ID',
       'SARS-CoV-2_PCR', 'Group', 'Annotation', 'assay_ontology_term_id',
       'cell_type_ontology_term_id', 'development_stage_ontology_term_id',
       'disease_ontology_term_id', 'ethnicity_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id',
       'tissue_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism',
       'sex', 'tissue', 'ethnicity', 'development_stage'],
      dtype='object')

In [7]:
# we have re annotated the cells a couple of times
meta = pd.read_csv("../data/current/celltype_abundance/global_all_ANNOTATED_CELLSTATES_RAW_V8_OBS_LONG.csv", index_col=0)
meta.Region = meta.Region.replace({"FW":"LV", "S":"LV", "AP":"LV"})
np.sum(meta.index == raw_obj.obs.index)

/home/icb/matthias.heinig/packages/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (8,9,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


881081

In [8]:
meta.columns

Index(['Patient', 'Sample', 'Sex', 'Gene', 'Phenotype',
       'Primary.Genetic.Phenotype', 'Age', 'Mutation.Type', 'Genomic.location',
       'Region', 'Tissue.source', 'n_genes', 'percent_mito', 'percent_ribo',
       'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo',
       'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log',
       'predicted_doublets_log', 'Replicate', 'cell_states_20200602',
       'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA',
       'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921',
       'cell_states_long_20200921', 'cell_states_20210218',
       'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03',
       'leiden_04', 'cell_type', 'cell_states_20210302', 'cell_states_long',
       'Assigned', 'cell_type_toplot1', 'cell_type_toplot2',
       'cell_type_toplot3', 'cell_type_toplot4', 'cell_states_20210510',
       'cell_states'],
      dtype='object')

In [9]:
meta.shape

(881081, 47)

## Define functions to update meta data with ontology terms

In [10]:
def update_uns(obj):
    #update uns
    obj.uns['version'] = {'corpora_encoding_version': '0.1.0', 'corpora_schema_version': '1.1.0'}
    obj.uns['title'] = 'DCM/ACM heart cell atlas'#Should be the publication title
    obj.uns['publication_doi'] = "no doi yet" #Publication DOI
    obj.uns['organism'] = 'Homo sapiens'
    obj.uns['organism_ontology_term_id'] = 'NCBITaxon:9606'
    obj.uns['layer_descriptions'] = {'X': 'log1p', 'raw.X': 'raw'}

In [11]:
def map_to_ontology(new_obs):
    nobs = new_obs.shape[0]#Set nobs for columns where all observations are the same
    
    ######
    # recode the age into bins of 10 years
    tens = np.round(new_obs['Age'] / 10) * 10
    new_obs.loc[:,'Age'] = pd.Series(tens)
    
    ######
    #tissue
    new_obs.loc[:,'tissue'] = pd.Series(['heart']).repeat(nobs).values
    new_obs.loc[:,'tissue_ontology_term_id'] = pd.Series(['UBERON:0000948']).repeat(nobs).values

    ######
    #assay
    new_obs.loc[:,'assay'] = pd.Series(['scRNA-seq']).repeat(nobs).values
    new_obs.loc[:,'assay_ontology_term_id'] = pd.Series(['EFO:0008913']).repeat(nobs).values

    ######
    #ethnicity
    new_obs.loc[:,'ethnicity'] = pd.Series(['unknown']).repeat(nobs).values
    new_obs.loc[:,'ethnicity_ontology_term_id'] = pd.Series(['']).repeat(nobs).values

    ######
    #development stage	
    new_obs.loc[:,'development_stage'] = pd.Series(['human adult stage']).repeat(nobs).values
    new_obs.loc[:,'development_stage_ontology_term_id'] = pd.Series(['HsapDv:0000087']).repeat(nobs).values

    ######
    #disease
    disease_mapping = {'DCM': 'dilated cardiomyopathy',
                       'ARVC': 'arrhythmogenic right ventricular cardiomyopathy',
                       'Control': 'normal',
                       'RCM/LVNC': 'Left Ventricular Non-Compaction Cardiomyopathy'}#appropriate mondo term?
    new_obs.loc[:,'disease'] = new_obs.loc[:,'Phenotype'].map(disease_mapping)

    disease_ontology_mapping = {'DCM': 'MONDO:0005021',
                                'ARVC': 'MONDO:0016587',
                                'Control': 'PATO:0000461',
                                'RCM/LVNC': 'HP:0011664'}#appropriate mondo term id?
    new_obs.loc[:,'disease_ontology_term_id'] = new_obs.loc[:,'Phenotype'].map(disease_ontology_mapping)

    ######
    #cell type
    cell_type_ontology_mapping = {'AD': 'CL:0000136',
                                  'CM': 'CL:0000746',
                                  'EC': 'CL:0000115',
                                  'FB': 'CL:0002548',
                                  'Lymphoid': 'CL:0000542',
                                  'Mast': 'CL:0000097',
                                  'Mural': 'CL:0008034',
                                  'Myeloid': 'CL:0000763',
                                  'NC': 'CL:0010022'}#appropriate CL term id?
    new_obs.loc[:,'cell_type_ontology_term_id'] = new_obs.loc[:,'cell_type'].map(cell_type_ontology_mapping)

    cell_type_mapping = {'AD': 'fat cell',
                         'CM': 'cardiac muscle cell',
                         'EC': 'endothelial cell',
                         'FB': 'fibroblast of cardiac tissue',
                         'Lymphoid': 'lymphocyte',
                         'Mast': 'mast cell',
                         'Mural': 'mural cell',
                         'Myeloid': 'myeloid cell',
                         'NC': 'neuronal cell'}#appropriate CL term?
    new_obs.loc[:,'cell_type'] = new_obs.loc[:,'cell_type'].map(cell_type_mapping)


    ######
    #sex
    sex_mapping = {'m': 'male',
                   'f': 'female'}

    new_obs.loc[:,'Sex'] = new_obs.loc[:,'Sex'].map(sex_mapping)
    return new_obs



Update the raw counts object

In [12]:
cols = ['Sample', 'Patient', 'Sex', 'Age', 'Region', 'Phenotype', 'Primary.Genetic.Phenotype', 
        'X10X_version', 'n_genes', 'n_counts', 'percent_mito', 'percent_ribo', 'scrublet_score_z', 
        'scrublet_score_log', 'solo_score', 'cell_type', 'cell_states', 'Assigned']

In [13]:
new_obs = meta.copy()
new_obs = map_to_ontology(new_obs.loc[:,cols])
new_obs.rename(columns={'Primary.Genetic.Phenotype':'Primary.Genetic.Diagnosis'}, inplace=True)
raw_obj.obs = new_obs
update_uns(raw_obj)

Test the updating with the adipocytes before looping over the rest of the objects

In [14]:
new_obj.X.shape

(2576, 33538)

In [15]:
new_obj.raw = raw_obj[new_obj.obs.index,:]

/home/icb/matthias.heinig/packages/miniconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


The endothelial cell object has a suffix that needs to be removed (see comment in code below). The mural object also has some barcodes that are not in the meta data. Check those:

In [16]:
obj = sc.read("%s/%s" % (input_dir, "Muralcells.h5ad"))
missing_bc = obj.obs.index[~obj.obs.index.isin(meta.index)].to_list()
missing_obs = obj.obs.loc[missing_bc,:]

In [17]:
missing_obs.loc[:,"Patient"].value_counts()

H84       0
H06       0
H02       0
H02_IC    0
H03       0
         ..
H33       0
H34       0
H35       0
H36       0
BS_H15    0
Name: Patient, Length: 79, dtype: int64

In [18]:
missing_obs.loc[missing_obs.loc[:,"Patient"].isin(["H15", "H42"]),:].head()

,Sample,Patient,Sex,Region,Phenotype,Primary.Genetic.Diagnosis,n_genes,n_counts,percent_mito,percent_ribo,scrublet_score_z,scrublet_score_log,solo_score,cell_type,cell_states


In [25]:
for ifile in input_files:
    print(ifile)
    obj = sc.read("%s/%s" % (input_dir, ifile))
    new_obs = meta.copy() #temp dataframe
    # check if endothelial cells and remove suffix
    if (ifile == "Endothelialcells.h5ad"):
        obj.obs.index=[re.sub("-[0-9]$", "", i) for i in obj.obs.index]
    # check if all barcodes are in the meta data table
    if (np.sum(obj.obs.index.isin(meta.index)) != obj.shape[0]):
        print("barcodes missing in meta data (found %s of %s)" % (np.sum(obj.obs.index.isin(meta.index)), obj.shape[0]))
        continue
    # check if all barcodes are in the raw data
    if (np.sum(obj.obs.index.isin(raw_obj.obs.index)) != obj.shape[0]):
        print("barcodes missing in raw data (found %s of %s)" % (np.sum(obj.obs.index.isin(raw_obj.obs.index)), obj.shape[0]))
        continue
    new_obs = new_obs.loc[obj.obs.index.to_list(),cols]
    new_obs = map_to_ontology(new_obs)
    new_obs.rename(columns={'Primary.Genetic.Phenotype':'Primary.Genetic.Diagnosis'}, inplace=True)
    update_uns(obj)
    obj.obs = new_obs
    # update the raw data
    obj.raw = raw_obj[new_obj.obs.index,:]
    ofile = "%s/%s" % (output_dir, ifile)
    print("writing output to %s" % ofile)
    obj.write(filename=ofile)

Cardiomyocytes2.h5ad


/home/icb/matthias.heinig/packages/miniconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/matthias.heinig/packages/miniconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])


writing output to ../data/current/publication/20220105/for_cellxgene/Cardiomyocytes2.h5ad


... storing 'Sample' as categorical
... storing 'Patient' as categorical
... storing 'Sex' as categorical
... storing 'Region' as categorical
... storing 'Phenotype' as categorical
... storing 'Primary.Genetic.Diagnosis' as categorical
... storing 'X10X_version' as categorical
... storing 'cell_type' as categorical
... storing 'cell_states' as categorical
... storing 'tissue' as categorical
... storing 'tissue_ontology_term_id' as categorical
... storing 'assay' as categorical
... storing 'assay_ontology_term_id' as categorical
... storing 'ethnicity' as categorical
... storing 'ethnicity_ontology_term_id' as categorical
... storing 'development_stage' as categorical
... storing 'development_stage_ontology_term_id' as categorical
... storing 'disease' as categorical
... storing 'disease_ontology_term_id' as categorical
... storing 'cell_type_ontology_term_id' as categorical


In [ ]:
ifile = input_files[0]
obj = sc.read("%s/%s" % (input_dir, ifile))

In [ ]:
np.sum(obj.obs.index.isin(meta.index)) == obj.shape[0]

In [ ]:
new_obs.rename(columns={'Primary.Genetic.Phenotype':'Primary.Genetic.Diagnosis'}, inplace=True)
new_obs.columns

In [ ]:
meta.index